In [10]:
from lunax.data_processing.utils import *
from lunax.models import xgb_reg
from lunax.hyper_opt import OptunaTuner

In [11]:
df_train = load_data('./example_data/regression/train.csv', 'csv')
df_train

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058
749996,749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242
749997,749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000
749998,749998,Style Guide,Episode 47,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,100.72939


In [12]:
target = 'Listening_Time_minutes'

In [13]:
df_train = preprocess_data(df_train, target)

In [14]:
X_train, X_val, y_train, y_val = split_data(df_train, target)

In [15]:
# 创建调参器 使用默认搜索空间
tuner = OptunaTuner(
    n_trials=3,
    model_class="XGBRegressor",
)

In [16]:
# 执行优化
results = tuner.optimize(X_train, y_train, X_val, y_val)
# 获取最优参数
best_params = results['best_params']

[I 2025-05-18 10:46:25,923] A new study created in memory with name: no-name-9a79d8aa-ca24-49dc-8d30-7be10c980469
[I 2025-05-18 10:46:42,145] Trial 0 finished with value: 169.04004584623894 and parameters: {'booster': 'gbtree', 'seed': 0, 'eta': 0.07697660707844997, 'lambda': 81.68914208335092, 'reg_lambda': 0.3965025117068618, 'reg_alpha': 70, 'gamma': 3.3444236609211098, 'subsample': 0.6133741554121142, 'max_depth': 12, 'colsample_bytree': 0.6000808431599329, 'min_child_weight': 1, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 169.04004584623894.
[I 2025-05-18 10:46:45,440] Trial 1 finished with value: 735.8300406602353 and parameters: {'booster': 'gblinear', 'seed': 0, 'eta': 0.08415993223037642, 'lambda': 6.283468533379233, 'reg_lambda': 0.5779604934593525, 'reg_alpha': 93}. Best is trial 0 with value: 169.04004584623894.
[I 2025-05-18 10:46:48,639] Trial 2 finished with value: 735.8300410570373 and parameters: {'booster': 'gblinear', 'seed': 0, 'eta': 0.0617441351458582

In [17]:
xgb_reg = xgb_reg(best_params)
xgb_reg.fit(X=X_train, y=y_train,k_fold=5)

[lunax]> Fold 1/5 - MSE: 169.3756, R2: 0.7700
[lunax]> Fold 2/5 - MSE: 171.5476, R2: 0.7677
[lunax]> Fold 3/5 - MSE: 169.6274, R2: 0.7701
[lunax]> Fold 4/5 - MSE: 171.9573, R2: 0.7658
[lunax]> Fold 5/5 - MSE: 171.1431, R2: 0.7676
[lunax]> Average scores - MSE: 170.7302, R2: 0.7682


In [18]:
xgb_reg.evaluate(X_val, y_val)

[lunax]> target value description:
+-------+--------+--------+-------+----------+
|   min |    max |   mean |   std |   median |
+=======+========+========+=======+==========+
|     0 | 119.97 |   45.4 | 27.13 |    43.36 |
+-------+--------+--------+-------+----------+
[lunax]> model evaluation results:
+-----------+--------+--------+-------+------+
| metrics   |   rmse |    mse |   mae |   r2 |
+===========+========+========+=======+======+
| values    |  13.03 | 169.84 |  9.52 | 0.77 |
+-----------+--------+--------+-------+------+


{'rmse': 13.032270494870719,
 'mse': 169.84007425147792,
 'mae': 9.522001283742041,
 'r2': 0.7691848807793433}